In [17]:
from typing import List
from pathlib import Path

import pandas as pd
import numpy as np

In [18]:
data_file_path = Path(r"..\Data\Agg_Workouts_2023.csv").resolve()
data = pd.read_csv(data_file_path)

In [32]:
sorted(data['cyclist_id'].unique())

[653,
 1054,
 1116,
 1304,
 2239,
 2434,
 2460,
 2581,
 2671,
 2748,
 3666,
 3982,
 4533,
 4865,
 5393,
 5428,
 5799,
 5832,
 5925,
 6163,
 6312,
 22629,
 22630,
 22631,
 22632,
 22633,
 22634,
 22635,
 22636]

In [20]:
from pandas.api.indexers import BaseIndexer
class BacwardForwardIndexer(BaseIndexer):
    """
    df = pd.DataFrame({"values": [0, 1, np.nan, 3, 4]})
    indexer = BacwardForwardIndexer(window_size=3)
    df.rolling(indexer).sum()
    --> 
        values
    0     1.0
    1     4.0
    2     8.0
    3     8.0
    4     8.0
    """
    def get_window_bounds(self, num_values, min_periods, center, closed, step):
        start = np.empty(num_values, dtype=np.int64)
        end = np.empty(num_values, dtype=np.int64)
        for i in range(num_values):
            start[i] = i - self.window_size
            end[i] = i + self.window_size
        return start, end

def rolling_mean(data: pd.DataFrame, columns: str | List[str], window_size: int) -> pd.DataFrame:
    def rolloing_mean_per_cyclist(rider: int):
        indexer = BacwardForwardIndexer(window_size=window_size)
        rider_col = data[data['cyclist_id'] == rider][columns]
        rolling_mean = rider_col.rolling(indexer, min_periods=1).mean()
        # fill na values with rolling mean
        data.loc[data['cyclist_id'] == rider, columns] = data.loc[data['cyclist_id'] == rider, columns].fillna(rolling_mean)
    
    for rider in data['cyclist_id'].unique():
        while data[data['cyclist_id'] == rider][columns].isna().sum().sum() > 0:
            rolloing_mean_per_cyclist(rider)

    return data

In [21]:
def fix_date(data: pd.DataFrame) -> pd.DataFrame:
    data['workout_datetime'] = pd.to_datetime(data['workout_datetime'])
    data['date'] = data['workout_datetime'].dt.date
    return data

def drop_cols_and_all_null(data: pd.DataFrame) -> pd.DataFrame:
    data.dropna(axis=1, how='all', inplace=True)
    cols_to_drop = ["workout_title", "workout_type", "workout_id", "workout_tp_id"]
    data.drop(columns=cols_to_drop, inplace=True)
    return data

def handle_missing_vals(data: pd.DataFrame) -> pd.DataFrame:
    data.loc[:, 'tss_calculation_method'] = data['tss_calculation_method'].fillna("Undefined")

    fill_w_zeros = ["elevation_gain", "elevation_loss", "elevation_average", "elevation_maximum", 
                    "elevation_minimum", "total_time", "distance", "calories", "IF", "tss_actual"]

    cols_to_roll = ["temp_avg", "temp_max", "temp_min"]

    for col in fill_w_zeros:
        data.loc[:, col] = data[col].fillna(0)

    data = rolling_mean(data, cols_to_roll, window_size=4)

    return data

In [22]:
data = drop_cols_and_all_null(data)
data = fix_date(data)
data = handle_missing_vals(data)

In [23]:
data

,cyclist_id,workout_datetime,workout_week,workout_month,elevation_gain,elevation_loss,elevation_average,elevation_maximum,elevation_minimum,temp_avg,temp_min,temp_max,total_time,distance,calories,IF,tss_actual,tss_calculation_method,date
0,653,2023-01-01 09:28:10,52,1,1830.0,1834.0,136.800003,342.000000,34.000000,9.309867,8.000000e+00,12.000000,5.118055,163312.093750,4377.0,0.692976,245.31,Power,2023-01-01
1,1116,2023-01-01 13:36:10,52,1,701.0,754.0,224.399994,350.399994,170.000000,25.316298,2.200000e+01,33.000000,2.983889,51553.980469,2238.0,0.562037,107.29,HeartRate,2023-01-01
2,2581,2023-01-01 13:11:06,52,1,418.0,419.0,61.599998,152.199997,4.000000,12.161635,1.100000e+01,19.000000,1.141389,28026.130859,641.0,0.471598,25.24,Power,2023-01-01
3,2671,2023-01-01 10:54:15,52,1,661.0,636.0,406.399994,744.599976,250.800003,13.359870,9.000000e+00,20.000000,2.183056,35485.160156,1333.0,0.600260,78.48,Power,2023-01-01
4,3666,2023-01-01 12:09:05,52,1,171.0,145.0,145.199997,188.000000,116.199997,13.448476,1.300000e+01,15.000000,1.146111,29830.650391,544.0,0.395429,17.85,Power,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8462,22633,2023-12-21 00:00:00,51,12,0.0,0.0,0.000000,0.000000,0.000000,17.666667,1.679944e-308,5.666667,0.000000,0.000000,0.0,0.000000,0.00,Undefined,2023-12-21
8463,22634,2023-12-21 11:24:05,51,12,1992.0,1994.0,1562.811035,1987.400024,1167.800049,6.569102,2.000000e+00,21.000000,2.984689,70.440609,2437.0,0.624865,116.37,Power,2023-12-21
8464,22635,2023-12-21 11:25:00,51,12,260.0,259.0,20.600000,56.000000,-5.600000,5.460807,5.000000e+00,13.000000,2.952500,88.408539,2100.0,0.559461,92.23,Power,2023-12-21
8465,22636,2023-12-21 00:00:00,51,12,0.0,0.0,0.000000,0.000000,0.000000,20.500000,2.674718e+59,10.500000,0.000000,0.000000,0.0,0.000000,0.00,Undefined,2023-12-21


In [24]:
def aggregate_workouts(data: pd.DataFrame) -> pd.DataFrame:
    """
    Some workouts occur on the same day, so we need to aggregate them first.
    Note: The data must be clean before passing it to this function.
    """
    agg_data = data.groupby(['cyclist_id', 'date']).agg(
        {
            'workout_week': 'first',
            'workout_month': 'first',
            'elevation_gain': 'sum',
            'elevation_loss': 'sum',
            'elevation_average': 'mean',
            'elevation_maximum': 'max',
            'elevation_minimum': 'min',
            'temp_avg': 'mean',
            'temp_min': 'min',
            'temp_max': 'max',
            'total_time': 'sum',
            'distance': 'sum',
            'calories': 'sum',
            'IF': 'mean',
            'tss_actual': 'sum',
            'tss_calculation_method': 'first'
        }
    ).reset_index()
    return agg_data

In [25]:
agg = aggregate_workouts(data)
agg[(agg['cyclist_id'] == 653) & (agg['date'] == pd.to_datetime('2023-01-12').date())]

,cyclist_id,date,workout_week,workout_month,elevation_gain,elevation_loss,elevation_average,elevation_maximum,elevation_minimum,temp_avg,temp_min,temp_max,total_time,distance,calories,IF,tss_actual,tss_calculation_method
11,653,2023-01-12,2,1,2007.0,2004.0,206.699997,493.399994,15.8,12.982998,11.0,17.0,3.983333,143592.078125,3496.0,0.711789,200.61,Power


In [26]:
data[(data['date'] == pd.to_datetime('2023-01-12').date()) & (data['cyclist_id'] == 653)]

,cyclist_id,workout_datetime,workout_week,workout_month,elevation_gain,elevation_loss,elevation_average,elevation_maximum,elevation_minimum,temp_avg,temp_min,temp_max,total_time,distance,calories,IF,tss_actual,tss_calculation_method,date
113,653,2023-01-12 10:04:44,2,1,912.0,915.0,216.199997,350.000000,86.400002,12.005608,11.0,15.0,1.883889,70889.671875,1749.0,0.751780,106.06,Power,2023-01-12
114,653,2023-01-12 12:23:51,2,1,1095.0,1089.0,197.199997,493.399994,15.800000,13.960388,12.0,17.0,2.099444,72702.406250,1747.0,0.671798,94.55,Power,2023-01-12


In [27]:
def add_missing_days(data: pd.DataFrame) -> pd.DataFrame:
    def add_missing_days_per_cyclist(rider: int) -> pd.DataFrame:
        rider_dates = data[data['cyclist_id'] == rider]['date']
        date_range = pd.date_range(start=rider_dates.min(), end=rider_dates.max()).date
        complete_df = pd.DataFrame({'date': date_range,
                                    'cyclist_id': [rider] * len(date_range)})
        merged_df = pd.merge(complete_df, data, on=['date', 'cyclist_id'], how='left')
        return merged_df
    
    return pd.concat([add_missing_days_per_cyclist(cyclist) for cyclist in data['cyclist_id'].unique()]).reset_index(drop=True)


def fix_week_and_month(data: pd.DataFrame) -> pd.DataFrame:
    dates = pd.to_datetime(data['date'])
    data['workout_week'] = dates.dt.isocalendar().week
    data['workout_month'] = dates.dt.month
    return data

In [28]:
agg = add_missing_days(agg)

In [29]:
rolled = fix_week_and_month(agg)
rolled = handle_missing_vals(rolled)

In [30]:
rolled.dtypes

date                       object
cyclist_id                  int64
workout_week               UInt32
workout_month               int32
elevation_gain            float64
elevation_loss            float64
elevation_average         float64
elevation_maximum         float64
elevation_minimum         float64
temp_avg                  float64
temp_min                  float64
temp_max                  float64
total_time                float64
distance                  float64
calories                  float64
IF                        float64
tss_actual                float64
tss_calculation_method     object
dtype: object